In [1]:
import numpy as np

# using truncated multivariate normal
# https://stackoverflow.com/questions/20115917/truncated-multivariate-normal-in-scipy
import emcee

In [ ]:
from numpy.linalg import inv, cholesky
from scipy.linalg import qr

def lnprob_trunc_norm(x, mean, bounds, C):
    if np.any(x < bounds[:,0]) or np.any(x > bounds[:,1]):
        return -np.inf
    else:
        return -0.5*(x-mean).dot(inv(C)).dot(x-mean)

In [ ]:
def multivariate_trunc_normal(mean, cov, bounds, size=1):
    ndim = mean.shape[0]
    nwalkers = max(10*ndim, size)
    nsteps = 1000 + size
    
    S = emcee.EnsembleSampler(nwalkers, ndim, lnprob_trunc_norm, args = (mean, bounds, cov))
    pos = emcee.utils.sample_ball(mean, np.sqrt(np.diag(C)), size=nwalkers)
    pos, prob, state = S.run_mcmc(pos, nsteps)
    
    return pos[-size:].flatten()

In [ ]:
ndim = 10
mean = (np.random.rand(ndim)-1)*3
bounds=np.ones((ndim, 2))
bounds[:, 0] = -3
bounds[:, 1] = 3
C = np.eye(ndim)
theta = multivariate_trunc_normal(mean, C, bounds)
theta

Now we will have a look at RJMCMC techniques here.

In this notebook, we will consider "AutoRJ" where the parameters can be assumed to follow some kind of random walk and the first and second moments are known. 

In [ ]:
mu1 = (np.random.rand(5)-1)*3
C1 = C[:5, :5]

B = cholesky(C)
B1 = cholesky(C1)

# now we sample u1 as standard normal.
u1 = np.random.normal(size=5)

In [ ]:
R, _ = qr(B)
rank_part = R.dot(inv(B)).dot((theta - mean).flatten())
theta1 = mu1 + B1.dot(rank_part[:B1.shape[0]])
# this is our new parameters for our model
theta1

In [ ]:
# if we assume that we will grow to 15 dimensions instead
mu1 = (np.random.rand(15)-1)*3
C1 = np.eye(15)
B1 = cholesky(C1)

# now we sample u1 as standard normal
u1 = np.random.normal(size=15)

In [ ]:
R, _ = qr(B1)

inner_part = inv(B).dot((theta - mean).flatten())
rank_part = R.dot(np.hstack([inner_part, mu1[-5:]]))

In [ ]:
theta1 = mu1 + B1.dot(rank_part[:B1.shape[0]])
# this is our new parameters for our model
theta1